In [1]:
!pip install transformers

In [2]:
!pip install bert_score

In [3]:
!pip install rouge

In [4]:
!pip install tensor2tensor && t2t-trainer




Traceback (most recent call last):
  File "/opt/conda/bin/t2t-trainer", line 33, in <module>
    tf.app.run(main)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/opt/conda/lib/python3.7/site-packages/absl/app.py", line 300, in run
    _run_main(main, args)
  File "/opt/conda/lib/python3.7/site-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "/opt/conda/bin/t2t-trainer", line 28, in main
    t2t_trainer.main(argv)
  File "/opt/conda/lib/python3.7/site-packages/tensor2tensor/bin/t2t_trainer.py", line 391, in main
    hparams = create_hparams()
  File "/opt/conda/lib/python3.7/site-packages/tensor2tensor/bin/t2t_trainer.py", line 184, in create_hparams
    hparams_path=hparams_path)
  File "/opt/conda/lib/python3.7/site-packages/tensor2tensor/utils/hparams_lib.py", line 48, in create_hparams
    hparams = registry.hparams(hpar

In [5]:
!pip install bunch

In [6]:
import tensorflow as tf
logical_devices = tf.config.list_logical_devices('CPU')
logical_devices

[LogicalDevice(name='/device:CPU:0', device_type='CPU')]

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
import tensorflow as tf
import os
tf.random.set_seed(100)

In [9]:

# core_path = '/content/drive/MyDrive/BERT_abstractive_summarization/abstractive_summarization'
core_path = os.getcwd()
dataset_name = 'cnn_dailymail'
vocab_path = os.path.join(core_path, 'input_files', 'vocab_files')
from bunch import Bunch

file_path_ = {
    'best_ckpt_path': os.path.join(core_path, 'created_files', 'training_summarization_model_ckpt', dataset_name, 'best_checkpoints'),
    'checkpoint_path': os.path.join(core_path, 'created_files', 'training_summarization_model_ckpt', dataset_name, 'checkpoints'),
    'vocab_path': os.path.join(core_path, 'input_files', 'Vocab_files'),
    'infer_csv_path': os.path.join(core_path, "input_files/Azure_dataset/Test.csv"),
    'infer_ckpt_path': os.path.join(core_path, 'created_files', 'training_summarization_model_ckpt', dataset_name, 'infer_checkpoints'),
    'log_path': os.path.join(core_path, "created_files/tensorflow.log"),
    'tensorboard_log': os.path.join(core_path, "created_files/tensorboard_logs/"+dataset_name+"/"),
    'summary_write_path': os.path.join(core_path, 'created_files', 'summaries', dataset_name),
    'subword_vocab_path': os.path.join(core_path, "input_files/vocab_file_summarization_"+dataset_name),
    'train_csv_path': os.path.join(core_path, "input_files/Azure_dataset/Train.csv"),
    'dataset': os.path.join(core_path, 'input_files', dataset_name)
}

file_path = Bunch(file_path_)



In [10]:
config_ = {

     'create_hist':False,             # create histogram of summary length and # of tokens per batch
     'doc_length': 512,
     'd_model': 768,                  # the projected word vector dimension
     'dff': 2048,                      # feed forward network hidden parameters
     'early_stop' : True,
     'eval_after' : 5000,              #Run evaluation after this many samples are trained 
     'init_tolerance':0,
     'input_vocab_size': 30522,        # total vocab size + start and end token
     'last_recorded_value': None,
     'monitor_metric' : 'combined_metric',
     'monitor_only_after': 1,        # monitor the monitor_metric only after this epoch                                           
     'max_tokens_per_line' : 1763,   # 1763 = 90 percentile  
     'num_examples_to_train': None,   #If None then all the examples in the dataset will be used to train
     'num_examples_to_infer': None,
     'num_heads': 8,                  # the number of heads in the multi-headed attention unit
     'num_layers': 8,                 # number of transformer blocks
     'print_chks': 50,                # print training progress per number of batches specified
     'pretrained_bert_model': 'bert-base-uncased',
     'run_tensorboard': True,
     'show_detokenized_samples' : False,
     'summ_length': 72,
     'start_from_batch':353556,
     'target_vocab_size': 30522,       # total vocab size + start and end token
     'test_size': 0.05,               # test set split sizeli
     'tfds_name' : 'cnn_dailymail',   # tfds dataset to be used
     'tolerance_threshold': 5,        # tolerance counter used for early stopping
     'use_tfds' : True,               # use tfds datasets as input to the model 
     'valid_samples_to_eval' : 100,
     'write_per_step': 5000,            # write summary for every specified epoch
     'write_summary_op': True         # write validation summary to hardisk
     }                                    

config = Bunch(config_)

#Parse log and get last_recorded_value of monitor_metric
try:
  with open(file_path.log_path) as f:
    for line in reversed(f.readlines()):
        if ('- tensorflow - INFO - '+ config.monitor_metric in line) and \
          (line[[i for i,char in enumerate((line)) if char.isdigit()][-1]+1] == '\n'):          
          config['last_recorded_value'] = float(line.split(config.monitor_metric)[1].split('\n')[0].strip())
          print(f"last_recorded_value of {config.monitor_metric} retained from last run {config['last_recorded_value']}")
#         if (' - tensorflow - INFO - Epoch 'in line):
#           last_batch_trained=line.split('Batch')[1].split('Train_Loss')[0]
#           config['start_from_batch']=int(last_batch_trained.strip())
#           print(f"Starting training from batch {config['start_from_batch']}")
          break
        else:
          continue
    if not config['last_recorded_value']:
      print('setting default value to last_recorded_value')
      config['last_recorded_value'] = 0 if config.monitor_metric != 'validation_loss' else float('inf')
#     if not config['start_from_batch']:
#       print('setting default value to start_from_batch')
#       config['start_from_batch'] = 0 
except FileNotFoundError:
  print('setting default value to last_recorded_value')
  #print('setting default value to start_from_batch')
  config['last_recorded_value'] = 0 if config.monitor_metric != 'validation_loss' else float('inf')
  #config['start_from_batch'] = 0 

setting default value to last_recorded_value


In [11]:
hyp = {
	 'accumulation_steps': 10,                                                                                   
	 'batch_size': 1,
	 'beam_sizes': [2, 3, 4],        	     # Used only during inference                                                 
	 'combined_metric_weights': [0.6, 0.4], #(bert_score, rouge, validation accuracy)
	 'dropout_rate': 0.0,
	 'epochs': 4,
	 'epsilon_ls': 0.0,              	     # label_smoothing hyper parameter
	 'grad_clipnorm':None,
	 'l2_norm':0,
	 'learning_rate': None,          	     # change to None to set learning rate decay
	 'length_penalty' : 1,                       # Beam search hyps . Used only during inference                                                 
	 'mean_attention_heads':True,                # if False then the attention parameters of the last head will be used
	 'mean_parameters_of_layers':True,           # if False then the attention parameters of the last layer will be used
	 'validation_batch_size' : 8
	 }                                    

h_parms = Bunch(hyp)

In [12]:
# -*- coding: utf-8 -*-
import datetime
import tensorflow as tf
import os
import logging
import tensorflow as tf

def check_and_create_dir(path):
    if not os.path.exists(path):
      os.makedirs(path)
      print(f'directory {path} created ')
        
# create folder in input_path if they don't exist
if not config.use_tfds:
  assert (os.path.exists(file_path['train_csv_path'])), 'Training dataset not available'
for key in file_path.keys():
  if key in ['infer_ckpt_path' , 'G_drive_vocab_path', 'subword_vocab_path']:
    pass
  else:
      if os.path.splitext(file_path[key])[1] == '':
        check_and_create_dir(file_path[key])
              

In [13]:
import logging
# get TF logger
log = logging.getLogger('tensorflow')
log.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh = logging.FileHandler(file_path.log_path)
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
log.addHandler(fh)
log.propagate = False

In [14]:



if not tf.config.experimental.list_physical_devices('GPU'):
    log.warning("GPU Not available so Running in CPU")

if config.run_tensorboard:
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = file_path.tensorboard_log + current_time + '/train'
    validation_log_dir = file_path.tensorboard_log + current_time + '/validation'
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    valid_summary_writer = tf.summary.create_file_writer(validation_log_dir)
else:
    train_summary_writer = None
    valid_summary_writer = None
        


In [15]:
# -*- coding: utf-8 -*-
import datetime
import tensorflow as tf
import os
import logging
import tensorflow as tf

# def check_and_create_dir(path):
#     if not os.path.exists(path):
#       os.makedirs(path)
#       print(f'directory {path} created ')
        
# # create folder in input_path if they don't exist
# if not config.use_tfds:
#   assert (os.path.exists(file_path['train_csv_path'])), 'Training dataset not available'
# for key in file_path.keys():
#   if key in ['infer_ckpt_path' , 'G_drive_vocab_path', 'subword_vocab_path']:
#     pass
#   else:
#       if os.path.splitext(file_path[key])[1] == '':
#         check_and_create_dir(file_path[key])
              
# # get TF logger
# log = logging.getLogger('tensorflow')
# log.setLevel(logging.DEBUG)
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# fh = logging.FileHandler(file_path.log_path)
# fh.setLevel(logging.DEBUG)
# fh.setFormatter(formatter)
# log.addHandler(fh)
# log.propagate = False

if not tf.config.experimental.list_physical_devices('GPU'):
    log.warning("GPU Not available so Running in CPU")

if config.run_tensorboard:
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = file_path.tensorboard_log + current_time + '/train'
    validation_log_dir = file_path.tensorboard_log + current_time + '/validation'
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    valid_summary_writer = tf.summary.create_file_writer(validation_log_dir)
else:
    train_summary_writer = None
    valid_summary_writer = None
        
# create metrics dict


In [16]:
from transformers import BertTokenizer
# Special Tokens
UNK_ID = 100
CLS_ID = 101
SEP_ID = 102
MASK_ID = 103

tokenizer = BertTokenizer.from_pretrained(config.pretrained_bert_model)

In [17]:
import shutil
import os
from rouge import Rouge
# from bert_score import score as b_score

log.info('Loading Pre-trained BERT model for BERT SCORE calculation')
# _, _, _ = b_score(["I'm Batman"], ["I'm Spiderman"], lang='en', model_type=config.pretrained_bert_model)
rouge_all = Rouge()

INFO:tensorflow:Loading Pre-trained BERT model for BERT SCORE calculation


In [18]:
import tensorflow as tf
import numpy as np

def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(
                          np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model
                          )
  
  # apply sin to even indices in the array; 2i
  sines = np.sin(angle_rads[:, 0::2])
  # apply cos to odd indices in the array; 2i+1
  cosines = np.cos(angle_rads[:, 1::2])
  pos_encoding = np.concatenate([sines, cosines], axis=-1)
  pos_encoding = pos_encoding[np.newaxis, ...]
  return tf.cast(pos_encoding, dtype=tf.float32)


def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions so that we can add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def create_masks(inp, tar):
  # Encoder padding mask
  enc_padding_mask = create_padding_mask(inp)  #output 1 if padded 0 is present else 0
  
  # Used in the 2nd attention block in the decoder.
  # This padding mask is used to mask the encoder outputs.
  dec_padding_mask = create_padding_mask(inp)
  # Used in the 1st attention block in the decoder.
  # It is used to pad and mask future tokens in the input received by 
  # the decoder.allows decoder to attend to all positions in the decoder up to and including that position(refer architecture)
  dec_target_padding_mask = create_padding_mask(tar)
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  return enc_padding_mask, combined_mask, dec_padding_mask

def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)  #(1 - lower_triangular_matrix)
  return mask  # (seq_len, seq_len)

def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.
    
    The mask is multiplied with -1e9 (close to negative infinity).
    This is done because the mask is summed with the scaled matrix
    multiplication of Q and K and is applied immediately before a softmax.
    The goal is to zero out these cells, and large negative inputs to softmax are near zero in the output.
    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.
    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_v, depth_v)

    return output, attention_weights

In [19]:
# class MultiHeadAttention(tf.keras.layers.Layer):
#   def __init__(self, d_model, num_heads):
#     super(MultiHeadAttention, self).__init__()
#     self.num_heads = num_heads
#     self.d_model = d_model
#     assert d_model % self.num_heads == 0, 'd_model should be a multiple of num_heads'
#     self.depth = d_model // self.num_heads
#     self.wq = tf.keras.layers.Dense(
#                                     d_model, 
#                                     kernel_regularizer = tf.keras.regularizers.l2(h_parms.l2_norm),
#                                     dtype='float32'
#                                     )
#     self.wk = tf.keras.layers.Dense(
#                                     d_model, 
#                                     kernel_regularizer = tf.keras.regularizers.l2(h_parms.l2_norm),
#                                     dtype='float32'
#                                     )
#     self.wv = tf.keras.layers.Dense(
#                                     d_model, 
#                                     kernel_regularizer = tf.keras.regularizers.l2(h_parms.l2_norm),
#                                     dtype='float32'
#                                     )
#     self.dense = tf.keras.layers.Dense(
#                                        d_model, 
#                                        kernel_regularizer = tf.keras.regularizers.l2(h_parms.l2_norm),
#                                        dtype='float32'
#                                        )
        
#   def split_heads(self, x, batch_size):
#     """Split the last dimension into (num_heads, depth).
#     Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
#     """
#     x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
#     return tf.transpose(x, perm=[0, 2, 1, 3])
    
#   def call(self, v, k, q, mask):
#     batch_size = tf.shape(q)[0]

#     q = self.wq(q)  # (batch_size, seq_len, d_model)
#     k = self.wk(k)  # (batch_size, seq_len, d_model)
#     v = self.wv(v)  # (batch_size, seq_len, d_model)

#     q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
#     k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
#     v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

#     # scaled_attention.shape == (batch_size, num_heads, seq_len_v, depth)
#     # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
#     scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)

#     scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_v, num_heads, depth)

#     concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))  # (batch_size, seq_len_v, d_model)

#     output = self.dense(concat_attention)  # (batch_size, seq_len_v, d_model)

#     return output, attention_weights

# arg1 (batch_size, seq_len, dff)
# arg2 (batch_size, seq_len, d_model)
def point_wise_feed_forward_network(d_model, dff):
  
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, 
                            activation='relu', 
                            kernel_regularizer = tf.keras.regularizers.l2(h_parms.l2_norm),
                            dtype='float32'),
      tf.keras.layers.Dense(d_model, 
                            kernel_regularizer = tf.keras.regularizers.l2(h_parms.l2_norm),
                            dtype='float32')
  ])



class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=h_parms.dropout_rate):
    super(DecoderLayer, self).__init__()

    self.mha1 = tf.keras.layers.MultiHeadAttention(key_dim=d_model, num_heads=num_heads)
    self.mha2 = tf.keras.layers.MultiHeadAttention(key_dim=d_model, num_heads=num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)
 
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6, dtype='float32')
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6, dtype='float32')
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6, dtype='float32')
    
    self.dropout1 = tf.keras.layers.Dropout(rate, dtype='float32')
    self.dropout2 = tf.keras.layers.Dropout(rate, dtype='float32')
    self.dropout3 = tf.keras.layers.Dropout(rate, dtype='float32')
    
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):
    
    
    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask, return_attention_scores=True)  
    attn1 = self.dropout1(attn1, training=training)
    x = tf.cast(x, tf.float32)
    out1 = self.layernorm1(attn1 + x)
    attn2, attn_weights_block2 = self.mha2(
      out1, enc_output, enc_output,  padding_mask, return_attention_scores=True)  
    attn2 = self.dropout2(attn2, training=training)
    # (batch_size, target_seq_len, d_model)
    out2 = self.layernorm2(attn2 + out1)  
    # (batch_size, target_seq_len, d_model)
    ffn_output = self.ffn(out2)  
    ffn_output = self.dropout3(ffn_output, training=training)
    # (batch_size, target_seq_len, d_model)
    out3 = self.layernorm3(ffn_output + out2)  
    return (out3, attn_weights_block1, attn_weights_block2)

class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, 
               rate=h_parms.dropout_rate):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    #self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model, dtype='float32')
    self.pos_encoding = positional_encoding(target_vocab_size, self.d_model)
    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate, dtype='float32')
    self.final_layer = tf.keras.layers.Dense(
                                             target_vocab_size, 
                                             dtype='float32',
                                             kernel_regularizer=tf.keras.regularizers.l2(h_parms.l2_norm),                                                      
                                             name='final_dense_layer'
                                             )
    
    
  def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
    seq_len = tf.shape(x)[1]
    attention_weights = {}
    # (batch_size, target_seq_len, d_model) 
    #x = self.embedding(x)  
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]
    x = self.dropout(x, training=training)
    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)
      attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
      attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
    # if h_parms.mean_parameters_of_layers:
    #   # take mean of the block 2 attention heads of all the layers
    #   block2_attention_weights = tf.reduce_mean(
    #                                             [(attention_weights[key]) for key in attention_weights.keys() if 'block2' in key], 
    #                                             axis=0
    #                                             )
    # else:
    #   # take the attention weights of the final layer 
    #   block2_attention_weights = attention_weights[f'decoder_layer{self.num_layers}_block2']
    

    # take mean of the block 2 attention heads of all the layers
    block2_attention_weights = tf.reduce_mean(
                                              [(attention_weights[key]) for key in attention_weights.keys() if 'block2' in key], 
                                              axis=0
                                              )
      
    # (batch_size, tar_seq_len, target_vocab_size)
    predictions = self.final_layer(tf.cast(x, tf.float32)) 
    
    # x (batch_size, target_seq_len, target_vocab_size)
    return predictions, block2_attention_weights


In [20]:
from transformers import TFBertModel
from tensorflow.keras.initializers import Constant

def tile_and_mask_diagonal(x, mask_with):
    """    
    Masks each word in the summary draft one by one with the [MASK] token
    At t-th time step the t-th word of input summary is
    masked, and the decoder predicts the refined word given other
    words of the summary.
    
    x :: (N, T)
    returrn :: (N, T-1, T)
    
    We do not mask the first and last postition (corresponding to [CLS]
    """

    N, T = tf.shape(x)[0], tf.shape(x)[1]

    first = tf.reshape(tf.tile(x[:, 0], [T-1]), [N, T-1, 1])
    
    x = x[:, 1:]
    T = T - 1
    
    masked = tf.reshape(tf.tile(x, [1, T]), [N, T, T])
    
    diag = tf.ones([N, T], dtype=masked.dtype) * mask_with
    masked = tf.linalg.set_diag(masked, diag)
    
    masked = tf.concat([first, masked], axis=2)
    
    masked = tf.reshape(masked, [N*T, T+1])
    
    return masked

def _embedding_from_bert():

  log.info("Extracting pretrained word embeddings weights from BERT")  
  vocab_of_BERT = TFBertModel.from_pretrained(config.pretrained_bert_model, trainable=False)
  embedding_matrix = vocab_of_BERT.get_weights()[0]
  log.info(f"Embedding matrix shape '{embedding_matrix.shape}'")
  return (embedding_matrix, vocab_of_BERT)

class AbstractiveSummarization(tf.keras.Model):
    """
    Pretraining-Based Natural Language Generation for Text Summarization 
    https://arxiv.org/pdf/1902.09243.pdf
    """
    def __init__(self, num_layers, d_model, num_heads, dff, vocab_size, output_seq_len, rate=0.1):
        super(AbstractiveSummarization, self).__init__()
        
        self.output_seq_len = output_seq_len
        self.vocab_size = vocab_size
        embedding_matrix, self.bert_model = _embedding_from_bert()
        self.embedding = tf.keras.layers.Embedding(
                                                    vocab_size, d_model, trainable=False,
                                                    embeddings_initializer=Constant(embedding_matrix)
                                                   )
        
        self.decoder = Decoder(num_layers, d_model, num_heads, dff, vocab_size, rate)
        self.d_model = d_model

    def draft_summary(self,
                      # input_ids,
                      enc_output,
                      look_ahead_mask,
                      padding_mask,
                      target_ids,
                      training):
        log.info("Building:'Draft summary'")
        # (batch_size, seq_len, d_bert)
        embeddings = self.embedding(target_ids) 
        # (batch_size, seq_len, vocab_len), (_)            
        draft_logits, draft_attention_dist = self.decoder(
                                                          # input_ids,
                                                          embeddings, 
                                                          enc_output, 
                                                          training, 
                                                          look_ahead_mask, 
                                                          padding_mask
                                                          )
        # (batch_size, seq_len, vocab_len)
        return draft_logits, draft_attention_dist

    def refine_summary(self,
                      #  input_ids, 
                       enc_output, 
                       target, 
                       padding_mask, 
                       training):
        log.info("Building:'Refined summary'")
        N = tf.shape(enc_output)[0]
        T = self.output_seq_len
        # (batch_size, seq_len) x3
        dec_inp_ids, dec_inp_mask, dec_inp_segment_ids = target
        # since we are using teacher forcing we do not need an autoregressice mechanism here
        # (batch_size x (seq_len - 1), seq_len) 
        dec_inp_ids = tile_and_mask_diagonal(dec_inp_ids, mask_with=MASK_ID)
        # (batch_size x (seq_len - 1), seq_len) 
        dec_inp_mask = tf.tile(dec_inp_mask, [T-1, 1])
        # (batch_size x (seq_len - 1), seq_len) 
        dec_inp_segment_ids = tf.tile(dec_inp_segment_ids, [T-1, 1])
        # (batch_size x (seq_len - 1), seq_len, d_bert) 
        enc_output = tf.tile(enc_output, [T-1, 1, 1])
        # (batch_size x (seq_len - 1), 1, 1, seq_len) 
        padding_mask = tf.tile(padding_mask, [T-1, 1, 1, 1])
        # (batch_size x (seq_len - 1), seq_len, d_bert)
        context_vectors = self.bert_model(dec_inp_ids)[0]

        # (batch_size x (seq_len - 1), seq_len, vocab_len), (_)
        dec_outputs, refine_attention_dist = self.decoder(
                                                          #  tf.tile(input_ids, [T-1, 1]),
                                                           context_vectors,
                                                           enc_output,
                                                           training,
                                                           look_ahead_mask=None,
                                                           padding_mask=padding_mask
                                                         )
        # (batch_size x (seq_len - 1), seq_len - 1, vocab_len)
        dec_outputs = dec_outputs[:, 1:, :]
        # (batch_size x (seq_len - 1), (seq_len - 1))
        diag = tf.linalg.set_diag(tf.zeros([T-1, T-1]), tf.ones([T-1]))
        diag = tf.tile(diag, [N, 1])
        
        where = tf.not_equal(diag, 0)
        indices = tf.where(where)
        
        # (batch_size x (seq_len - 1), vocab_len)
        dec_outputs = tf.gather_nd(dec_outputs, indices)
        
        # (batch_size, seq_len - 1, vocab_len)
        dec_outputs = tf.reshape(dec_outputs, [N, T-1, -1])
        # (batch_size, seq_len, vocab_len)
        refine_logits = tf.concat(
                           [tf.tile(tf.expand_dims(tf.one_hot([CLS_ID], self.vocab_size), axis=0), [N, 1, 1]), dec_outputs],
                           axis=1
                           )


        # (batch_size, seq_len, vocab_len)
        return refine_logits, refine_attention_dist

    def call(self, 
             input_ids, 
             input_mask, 
             input_segment_ids, 
             target_ids, 
             target_mask, 
             target_segment_ids, 
             training):

           # (batch_size, 1, 1, seq_len), (batch_size, 1, 1, seq_len)
        _, combined_mask, dec_padding_mask = create_masks(input_ids, target_ids[:, :-1])

        # (batch_size, seq_len, d_bert)
        enc_output = self.bert_model(input_ids)[0]

        # (batch_size, seq_len, vocab_len), _
        draft_logits, draft_attention_dist = self.draft_summary(
                                                                # input_ids,
                                                                enc_output=enc_output,
                                                                look_ahead_mask=combined_mask,
                                                                padding_mask=dec_padding_mask,
                                                                target_ids=target_ids[:, :-1],
                                                                training=True
                                                               )

        # (batch_size, seq_len, vocab_len), _
        refine_logits, refine_attention_dist = self.refine_summary(
                                                                  # input_ids,
                                                                  enc_output=enc_output,
                                                                  target=(target_ids[:, :-1], 
                                                                          target_mask[:, :-1], 
                                                                          target_segment_ids[:, :-1]),            
                                                                  padding_mask=dec_padding_mask,
                                                                  training=True
                                                                  )
              
        return draft_logits, draft_attention_dist, refine_logits, refine_attention_dist

In [21]:
# Metrics
with tf.device('/cpu:0'):
    monitor_metrics = dict()
    monitor_metrics['validation_loss'] = None
    # monitor_metrics['BERT_f1'] = None
    monitor_metrics['ROUGE_f1'] = None
    monitor_metrics['validation_accuracy'] = None
    monitor_metrics['combined_metric'] = (
    #                                       monitor_metrics['BERT_f1'], 
                                          monitor_metrics['ROUGE_f1'], 
                                          monitor_metrics['validation_accuracy']
                                          )
    assert (config.monitor_metric in monitor_metrics.keys()), f'Available metrics to monitor are {monitor_metrics.keys()}'
    assert (tf.reduce_sum(h_parms.combined_metric_weights) == 1), 'weights should sum to 1'
    
    class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
      def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)
        self.warmup_steps = warmup_steps

      def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


    def label_smoothing(inputs, epsilon=0.1):
        # number of channels
        V = inputs.get_shape().as_list()[-1] 
        # epsilon = tf.cast(epsilon, dtype=inputs.dtype)
        # V = tf.cast(V, dtype=inputs.dtype)
        return ((1-epsilon) * inputs) + (epsilon / V)

    def convert_wordpiece_to_words(w_piece):
      new=[]
      for i in w_piece:
        if '##' in i:
          m = i.replace('##', '')
        else:
          if w_piece.index(i) == 0:
            m = i
          else:
            m = ' '+i
        new.append(m)
      return (''.join(new))

    def loss_function(real, pred, mask):
      loss_object = tf.keras.losses.CategoricalCrossentropy(
                                                          from_logits=True, 
                                                          reduction='none'
                                                          )
      # pred shape == real shape = (batch_size, tar_seq_len, target_vocab_size)
      loss_ = loss_object(real, pred)
      mask = tf.cast(mask, dtype=loss_.dtype)
      loss_ *= mask
      return loss_

    def get_loss_and_accuracy():
        loss = tf.keras.metrics.Mean()
        accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
        return(loss, accuracy)

    def write_summary(tar_real, predictions, step, write=config.write_summary_op):
      r_avg_final = []
      total_summary = []
      for i, sub_tar_real in enumerate(tar_real):
        predicted_id = tf.cast(tf.argmax(predictions[i], axis=-1), tf.int32)
        #decoded_logits = argmax(predictions[i])
        #predicted_id = tf.cast(decoded_logits, tf.int32)
        sum_ref = tokenizer.convert_ids_to_tokens([i for i in sub_tar_real.numpy() if i not in [0, 101, 102]])
        sum_hyp = tokenizer.convert_ids_to_tokens([i for i in predicted_id.numpy() if i not in [0, 101, 102]])
        sum_ref = convert_wordpiece_to_words(sum_ref)
        sum_hyp = convert_wordpiece_to_words(sum_hyp)
        # don't consider empty values for ROUGE and BERT score calculation
        if sum_hyp and sum_ref:
          total_summary.append((sum_ref, sum_hyp))
      ref_sents = [ref for ref, _ in total_summary]
      hyp_sents = [hyp for _, hyp in total_summary]
      # returns :- dict of dicts
      if ref_sents and hyp_sents:
          try:  
            rouges = rouge_all.get_scores(ref_sents , hyp_sents)
            avg_rouge_f1 = np.mean([np.mean([rouge_scores['rouge-1']["f"], rouge_scores['rouge-2']["f"], rouge_scores['rouge-l']["f"]]) for rouge_scores in rouges])
            # _, _, bert_f1 = b_score(ref_sents, hyp_sents, lang='en', model_type=config.pretrained_bert_model)
            rouge_score =  avg_rouge_f1.astype('float64')
#             bert_f1_score =  np.mean(bert_f1.tolist(), dtype=np.float64)
          except ValueError:
            log.warning('Problem in calculating the ROUGE scores')
            rouge_score = 0
#             bert_f1_score = 0
      else:
          log.warning('The sentences predicted by the model are empty so setting the scores to 0')
          rouge_score = 0
#           bert_f1_score = 0

      if write and (step)%config.write_per_step == 0:
        with tf.io.gfile.GFile(file_path.summary_write_path+str(step.numpy()), 'w') as f:
          for ref, hyp in total_summary:
            f.write(ref+'\t'+hyp+'\n')
      return rouge_score


    def tf_write_summary(tar_real, predictions, step):
      return tf.py_function(write_summary, [tar_real, predictions, step], Tout=[tf.float32])

    # create metrics dict
    monitor_metrics = dict()
    monitor_metrics['validation_loss'] = None
#     monitor_metrics['BERT_f1'] = None
    monitor_metrics['ROUGE_f1'] = None
    monitor_metrics['validation_accuracy'] = None
    monitor_metrics['combined_metric'] = (
#                                           monitor_metrics['BERT_f1'], 
                                          monitor_metrics['ROUGE_f1'], 
                                          monitor_metrics['validation_accuracy']
                                          )
    assert (config.monitor_metric in monitor_metrics.keys()), f'Available metrics to monitor are {monitor_metrics.keys()}'
    assert (tf.reduce_sum(h_parms.combined_metric_weights) == 1), 'weights should sum to 1'

    def monitor_run(latest_ckpt, 
                    ckpt_save_path, 
                    val_loss, 
                    val_acc,
                    bert_score, 
                    rouge_score, 
                    valid_summary_writer,
                    step,
                    to_monitor=config.monitor_metric):

      ckpt_fold, ckpt_string = os.path.split(ckpt_save_path)
      if config.run_tensorboard:
        with valid_summary_writer.as_default():
          tf.summary.scalar('validation_total_loss', val_acc, step=step)
          tf.summary.scalar('validation_total_accuracy', val_loss, step=step)
          tf.summary.scalar('ROUGE_f1', rouge_score, step=step)
#           tf.summary.scalar('BERT_f1', bert_score, step=step)
      monitor_metrics = dict()
      monitor_metrics['validation_loss'] = val_loss
      monitor_metrics['validation_accuracy'] = val_acc
      # monitor_metrics['BERT_f1'] = bert_score
      monitor_metrics['ROUGE_f1'] = rouge_score
      monitor_metrics['combined_metric'] = (
                                            # monitor_metrics['BERT_f1'], 
                                            monitor_metrics['ROUGE_f1'], 
                                            monitor_metrics['validation_accuracy']
                                            )
      # multiply with the weights                                    
      monitor_metrics['combined_metric'] = round(tf.reduce_sum([(i*j) for i,j in zip(monitor_metrics['combined_metric'],  
                                                                                     h_parms.combined_metric_weights)]).numpy(), 2)
      log.info(f"combined_metric {monitor_metrics['combined_metric']:4f}")
      if to_monitor != 'validation_loss':
        cond = (config.last_recorded_value < monitor_metrics[to_monitor])
      else:
        cond = (config.last_recorded_value > monitor_metrics[monitor])
      if (latest_ckpt > config.monitor_only_after) and cond:
        # reset tolerance to zero if the monitor_metric decreases before the tolerance threshold
        config.init_tolerance=0
        config.last_recorded_value =  monitor_metrics[to_monitor]
        ckpt_files_tocopy = [files for files in os.listdir(os.path.split(ckpt_save_path)[0]) \
                             if ckpt_string in files]
        log.info(f'{to_monitor} is {monitor_metrics[to_monitor]:4f} so checkpoint files {ckpt_string}           \
                 will be copied to best checkpoint directory')
        # copy the best checkpoints
        shutil.copy2(os.path.join(ckpt_fold, 'checkpoint'), file_path.best_ckpt_path)
        for files in ckpt_files_tocopy:
            shutil.copy2(os.path.join(ckpt_fold, files), file_path.best_ckpt_path)
      else:
        config.init_tolerance+=1
      # Warn and early stop
      if config.init_tolerance > config.tolerance_threshold:
        log.warning('Tolerance exceeded')
      if config.early_stop and config.init_tolerance > config.tolerance_threshold:
        log.info(f'Early stopping since the {to_monitor} reached the tolerance threshold')
        return False
      else:
        return True

    





In [22]:
!pip install sklearn

In [23]:
# Create dataset

import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
from functools import partial
with tf.device('/cpu:0'):
    def create_dataframe(path, num_examples):
        df = pd.read_csv(path)
        df.columns = [i.capitalize() for i in df.columns if i.lower() in ['document', 'summary']]
        assert len(df.columns) == 2, 'column names should be document and summary'
        df = df[:num_examples]
        assert not df.isnull().any().any(), 'dataset contains  nans'
        return (df["Document"].values, df["Summary"].values)

    def pad(l, n, pad=0):
        """
        Pad the list 'l' to have size 'n' using 'padding_element'
        """
        pad_with = (0, max(0, n - len(l)))
        return np.pad(l, pad_with, mode='constant', constant_values=pad)


    def encode(sent_1, sent_2, tokenizer, input_seq_len, output_seq_len):
        """
        Encode the text to the BERT expected format

        'input_seq_len' is used to truncate the the article length
        'output_seq_len' is used to truncate the the summary length
        BERT has the following special tokens:    

        [CLS] : The first token of every sequence. A classification token
        which is normally used in conjunction with a softmax layer for classification
        tasks. For anything else, it can be safely ignored.
        [SEP] : A sequence delimiter token which was used at pre-training for
        sequence-pair tasks (i.e. Next sentence prediction). Must be used when
        sequence pair tasks are required. When a single sequence is used it is just appended at the end.
        [MASK] : Token used for masked words. Only used for pre-training.

        Additionally BERT requires additional inputs to work correctly:
            - Mask IDs
            - Segment IDs

        The mask has 1 for real tokens and 0 for padding tokens. Only real tokens are attended to.
        Sentence Embeddings is just a numeric class to distinguish between pairs of sentences.
        """
        input_ids_1 = tokenizer.encode(sent_1.numpy().decode('utf-8'))
        input_ids_2 = tokenizer.encode(sent_2.numpy().decode('utf-8'))

        # Account for [CLS] and [SEP] with "- 2"
        if len(input_ids_1) > input_seq_len - 2:
            input_ids_1 = input_ids_1[0:(input_seq_len - 2)]
        if len(input_ids_2) > (output_seq_len + 1) - 2:
            input_ids_2 = input_ids_2[0:((output_seq_len + 1) - 2)]

        input_mask_1 = [1] * len(input_ids_1)
        input_mask_2 = [1] * len(input_ids_2)

        input_ids_1 = pad(input_ids_1, input_seq_len, 0)
        input_ids_2 = pad(input_ids_2, output_seq_len + 1, 0)
        input_mask_1 = pad(input_mask_1, input_seq_len, 0)
        input_mask_2 = pad(input_mask_2, output_seq_len + 1, 0)

        input_type_ids_1 = [0] * len(input_ids_1)
        input_type_ids_2 = [0] * len(input_ids_2)

        return input_ids_1, input_mask_1, input_type_ids_1, input_ids_2, input_mask_2, input_type_ids_2


    def tf_encode(tokenizer, input_seq_len, output_seq_len):
        """
        Operations inside `.map()` run in graph mode and receive a graph
        tensor that do not have a `numpy` attribute.
        The tokenizer expects a string or Unicode symbol to encode it into integers.
        Hence, you need to run the encoding inside a `tf.py_function`,
        which receives an eager tensor having a numpy attribute that contains the string value.
        """    
        def f(s1, s2):
            encode_ = partial(encode, tokenizer=tokenizer, input_seq_len=input_seq_len, output_seq_len=output_seq_len)
            return tf.py_function(encode_, [s1, s2], [tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32])

        return f

    # Set threshold for document and  summary length
    def filter_max_length(x, x1, x2, y, y1, y2):
        return tf.logical_and(
                              tf.size(x[0]) <= config.doc_length,
                              tf.size(y[0]) <= config.summ_length
                             )

    def filter_combined_length(x, x1, x2, y, y1, y2):
        return tf.math.less_equal(
                                  (tf.math.count_nonzero(x) + tf.math.count_nonzero(y)), 
                                  config.max_tokens_per_line
                                 )

    # this function should be added after padded batch step
    def filter_batch_token_size(x, x1, x2, y, y1, y2):
        return tf.math.less_equal(
                                  (tf.size(x[0]) + tf.size(y[0])), 
                                  config.max_tokens_per_line*h_parms.batch_size
                                 )

    def map_batch_shuffle(dataset, buffer_size, split, 
                          shuffle=True, batch_size=h_parms.batch_size,
                          filter_off=False):
        # with tf.device('/cpu:0'):
        tf_dataset = dataset.map(
                                tf_encode(
                                    tokenizer, 
                                    512, 
                                    72
                                    ), num_parallel_calls=1
                                )
        if not filter_off:
            tf_dataset = tf_dataset.filter(filter_combined_length)

        if split == 'train' and shuffle and (not config.use_tfds):
          tf_dataset = tf_dataset.shuffle(buffer_size)
        tf_dataset = tf_dataset.padded_batch(batch_size, padded_shapes=([-1], [-1], [-1], [-1], [-1], [-1]))
        tf_dataset = tf_dataset.prefetch(1)
        # tf_dataset = tf_dataset.cache()
        return tf_dataset

    def create_train_data(num_samples_to_train = config.num_examples_to_train, shuffle=True, filter_off=True):

        if config.use_tfds:
            examples, metadata = tfds.load(
                                           config.tfds_name, 
                                           with_info=True,
                                           as_supervised=True, 
                                           data_dir='cnn_dataset',
                                           builder_kwargs={"version": "3.0.0"}
                                          )
            other_ds = 'validation' if 'validation' in examples else 'test'
            train_examples = examples['train']
            valid_examples = examples[other_ds]
            train_examples = train_examples.take(100)
            valid_examples = valid_examples.take(100)
            # train_buffer_size = metadata.splits['train'].num_examples
            train_buffer_size = len(train_examples)
            # valid_buffer_size = metadata.splits[other_ds].num_examples
            valid_buffer_size = len(valid_examples)
        else:
            doc, summ = create_dataframe(file_path.train_csv_path, num_samples_to_train)
            X_train, X_test, y_train, y_test = train_test_split(
                                                                doc, 
                                                                summ, 
                                                                test_size=config.test_size, 
                                                                random_state=42
                                                                )
            train_examples = tf.data.Dataset.from_tensor_slices((X_train, y_train))
            valid_examples = tf.data.Dataset.from_tensor_slices((X_test, y_test))
            train_buffer_size = len(X_train) 
            valid_buffer_size = len(X_test)
        train_dataset = map_batch_shuffle(
                                         train_examples, 
                                         train_buffer_size, 
                                         split = 'train',
                                         shuffle = shuffle,
                                         batch_size=h_parms.batch_size,
                                         filter_off=filter_off
                                         )
        valid_dataset = map_batch_shuffle(
                                         valid_examples, 
                                         valid_buffer_size, 
                                         split='valid',
                                         batch_size=h_parms.validation_batch_size,
                                         filter_off=filter_off
                                         )
        log.info('Train and Test tf_datasets created')
        return (train_dataset, valid_dataset, train_buffer_size, valid_buffer_size)

    def infer_data_from_df(num_of_infer_examples=config.num_examples_to_infer):
        doc, summ = create_dataframe(file_path.infer_csv_path, num_of_infer_examples)
        infer_examples = tf.data.Dataset.from_tensor_slices((doc, summ))
        infer_buffer_size = len(doc)
        infer_dataset = map_batch_shuffle(
                                          infer_examples, 
                                          infer_buffer_size, 
                                          split = 'infer',
                                          batch_size=1              
                                          )
        log.info('infer tf_dataset created')
        return infer_dataset

In [24]:

model_metrics = 'Step {}\n,\
                 Train Loss {:.4f}\n,\
                 Train_Accuracy {:.4f}\n,\
                 validation_loss {:.4f}\n,\
                 validation_accuracy {:4f}\n,\
                 ROUGE_f1 {:4f}\n,\
                 BERT_f1 {:4f}\n'
evaluation_step  = 'Time taken for {} step : {} secs' 
checkpoint_details = 'Saving checkpoint at step {} on {}'
batch_zero = 'Time taken to feed the input data to the model {} seconds'
batch_run_details = 'Step {} Train_Loss {:.4f} Train_Accuracy {:.4f}'

# run every batch
with tf.device('/cpu:0'):
    def batch_run_check(batch, start, train_summary_writer, train_loss, train_accuracy, model):
      if config.run_tensorboard:
        with train_summary_writer.as_default():
          tf.summary.scalar('train_loss', train_loss, step=batch)
          tf.summary.scalar('train_accuracy', train_accuracy, step=batch)
      if batch==0:
        log.info(model.summary())
        log.info(batch_zero.format(time.time()-start))
      log.info(
               batch_run_details.format(
                                       batch, 
                                       train_loss, 
                                       train_accuracy
                                       )
              )

    # run after each epoch
    def count_recs(batch, epoch, num_of_train_examples):
      if epoch == 0:
        try:
          if batch > 0:
            num_of_recs_post_filter_atmost = ((batch)*h_parms.batch_size)/num_of_train_examples
            log.info(f'Percentage of records used for training should be close to {num_of_recs_post_filter_atmost*100 :.2f}')
        except NameError:
          log.info('End of epoch')

    def calc_validation_loss(validation_dataset, 
                             step, 
                             val_step, 
                             valid_summary_writer, 
                             validation_loss, 
                             validation_accuracy):
      total_val_acc_avg = tf.keras.metrics.Mean()
      total_val_loss_avg = tf.keras.metrics.Mean()
      for (batch, (input_ids, input_mask, input_segment_ids, target_ids_, target_mask, target_segment_ids)) in enumerate(validation_dataset.take(config.valid_samples_to_eval//h_parms.validation_batch_size)):
        # calculate rouge for only the first batch
        if batch == 0:
          draft_mask = tf.math.logical_not(tf.math.equal(target_ids_[:, 1:], 0))
          refine_mask = tf.math.logical_not(tf.math.equal(target_ids_[:, :-1], 0))
          target_ids = label_smoothing(tf.one_hot(target_ids_, depth=config.input_vocab_size))
          rouge_score, bert_score = val_step(input_ids, 
                                             input_mask, 
                                             input_segment_ids, 
                                             target_ids_, 
                                             target_mask, 
                                             target_segment_ids, 
                                             target_ids, 
                                             draft_mask, 
                                             refine_mask, 
                                             step, 
                                             config.write_summary_op
                                             )
        else:
          _  =  val_step(input_ids, 
                         input_mask, 
                         input_segment_ids, 
                         target_ids_, 
                         target_mask, 
                         target_segment_ids, 
                         target_ids, 
                         draft_mask, 
                         refine_mask,
                         step, 
                         False
                         )
        if config.run_tensorboard:
          with valid_summary_writer.as_default():
            tf.summary.scalar('validation_loss', validation_loss.result(), step=batch)
            tf.summary.scalar('validation_accuracy', validation_accuracy.result(), step=batch)
      return (total_val_acc_avg(validation_accuracy.result()), 
              total_val_loss_avg(validation_loss.result()), 
              rouge_score, 
              bert_score)

In [25]:
with tf.device('/cpu:0'):
    model = AbstractiveSummarization(
                                  num_layers=config.num_layers, 
                                  d_model=config.d_model, 
                                  num_heads=config.num_heads, 
                                  dff=config.dff, 
                                  vocab_size=config.input_vocab_size,
                                  output_seq_len=config.summ_length, 
                                  rate=h_parms.dropout_rate
                                  )

INFO:tensorflow:Extracting pretrained word embeddings weights from BERT


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


INFO:tensorflow:Embedding matrix shape '(30522, 768)'


In [26]:
from __future__ import absolute_import, division, print_function, unicode_literals
import time
import tensorflow as tf


with tf.device('/cpu:0'):
    train_dataset, val_dataset, num_of_train_examples, num_of_val_examples = create_train_data()



    lr = h_parms.learning_rate if h_parms.learning_rate else CustomSchedule(config.d_model)    

    if h_parms.grad_clipnorm:
      optimizer = tf.keras.optimizers.Adam(
                                 learning_rate=lr, 
                                 beta_1=0.9, 
                                 beta_2=0.98, 
                                 clipnorm=h_parms.grad_clipnorm,
                                 epsilon=1e-9
                                 )
    else:
      optimizer = tf.keras.optimizers.Adam(
                                 learning_rate=lr, 
                                 beta_1=0.9, 
                                 beta_2=0.98, 
                                 epsilon=1e-9
                                 )

    log.info(f'{num_of_train_examples} training samples')
    log.info(f'{num_of_val_examples} validation samples')
    train_loss, train_accuracy = get_loss_and_accuracy()
    validation_loss, validation_accuracy = get_loss_and_accuracy()
    accumulators = []

    train_step_signature = [
                          tf.TensorSpec(shape=(None, None), dtype=tf.int32),
                          tf.TensorSpec(shape=(None, None), dtype=tf.int32),
                          tf.TensorSpec(shape=(None, None), dtype=tf.int32),
                          tf.TensorSpec(shape=(None, None), dtype=tf.int32),
                          tf.TensorSpec(shape=(None, None), dtype=tf.int32),
                          tf.TensorSpec(shape=(None, None), dtype=tf.int32),
                          tf.TensorSpec(shape=(None, None, None), dtype=tf.float32),
                          tf.TensorSpec(shape=(None, None), dtype=tf.bool),
                          tf.TensorSpec(shape=(None, None), dtype=tf.bool),
                          tf.TensorSpec(shape=(None), dtype=tf.bool)
                          ]

    val_step_signature = [
                          tf.TensorSpec(shape=(None, None), dtype=tf.int32),
                          tf.TensorSpec(shape=(None, None), dtype=tf.int32),
                          tf.TensorSpec(shape=(None, None), dtype=tf.int32),
                          tf.TensorSpec(shape=(None, None), dtype=tf.int32),
                          tf.TensorSpec(shape=(None, None), dtype=tf.int32),
                          tf.TensorSpec(shape=(None, None), dtype=tf.int32),
                          tf.TensorSpec(shape=(None, None, None), dtype=tf.float32),
                          tf.TensorSpec(shape=(None, None), dtype=tf.bool),
                          tf.TensorSpec(shape=(None, None), dtype=tf.bool),
                          tf.TensorSpec(shape=(None), dtype=tf.int32),
                          tf.TensorSpec(shape=(None), dtype=tf.bool)
                         ]
    def check_ckpt(checkpoint_path):
        ckpt = tf.train.Checkpoint(
                                   model=model,
                                   optimizer=optimizer
                                  )
        ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=10)
        if tf.train.latest_checkpoint(checkpoint_path):
          ckpt.restore(ckpt_manager.latest_checkpoint)
          log.info(ckpt_manager.latest_checkpoint +' restored')
          latest_ckpt = int(ckpt_manager.latest_checkpoint[-2:])
        else:
            latest_ckpt=0
            log.info('Training from scratch')
        return (ckpt_manager, latest_ckpt)
    # if a checkpoint exists, restore the latest checkpoint.
    





# total_steps = int(h_parms.epochs * (h_parms.accumulation_steps))
# train_dataset = train_dataset.repeat(total_steps)


INFO:absl:No config specified, defaulting to first: cnn_dailymail/plain_text
INFO:absl:Load dataset info from cnn_dataset/cnn_dailymail/plain_text/3.0.0
INFO:absl:Reusing dataset cnn_dailymail (cnn_dataset/cnn_dailymail/plain_text/3.0.0)
INFO:absl:Constructing tf.data.Dataset for split None, from cnn_dataset/cnn_dailymail/plain_text/3.0.0


INFO:tensorflow:Train and Test tf_datasets created
INFO:tensorflow:100 training samples
INFO:tensorflow:100 validation samples


In [27]:
@tf.function(input_signature=train_step_signature)  
def train_step(input_ids, 
               input_mask, 
               input_segment_ids, 
               target_ids_, 
               target_mask, 
               target_segment_ids, 
               target_ids, 
               draft_mask, 
               refine_mask,
               grad_accum_flag):
  with tf.GradientTape() as tape:
    with tf.device('/cpu:0'):
        log.info(f'Calculate draft_predictions, refine_predictions')
        (draft_predictions, draft_attention_weights, refine_predictions, refine_attention_weights) = model(
                                                               input_ids, input_mask, input_segment_ids, 
                                                               target_ids_, target_mask, target_segment_ids, 
                                                               True
                                                               )
    train_variables = model.trainable_variables
    log.info(f'draft_predictions shape: {draft_predictions.shape} - draft_mask shape: {draft_mask.shape}')
    draft_summary_loss = loss_function(target_ids[:, 1:, :], draft_predictions, draft_mask)
    log.info(f'refine_predictions shape: {refine_predictions.shape} - refine_mask shape: {refine_mask.shape}')
    refine_summary_loss = loss_function(target_ids[:, :-1, :], refine_predictions, refine_mask)
    loss = draft_summary_loss + refine_summary_loss
    loss = tf.reduce_mean(loss)
    
    #loss = optimizer.get_scaled_loss(loss)
  gradients  = tape.gradient(loss, train_variables)
  #gradients = optimizer.get_unscaled_gradients(gradients)
  # Initialize the shadow variables with same type as the gradients 
  if not accumulators:
    for tv in gradients:
      accumulators.append(tf.Variable(tf.zeros_like(tv), trainable=False))
  # accmulate the gradients to the shadow variables
  for (accumulator, grad) in zip(accumulators, gradients):
    accumulator.assign_add(grad)
  # apply the gradients and reset them to zero if the flag is true
  if grad_accum_flag:
    optimizer.apply_gradients(zip(accumulators, train_variables))
    for accumulator in (accumulators):
        accumulator.assign(tf.zeros_like(accumulator))
  
    train_loss(loss)
    train_accuracy(target_ids_[:, :-1], refine_predictions)  
  return (target_ids_[:, :-1], refine_predictions)

In [28]:
# @tf.function(input_signature=val_step_signature)
# def val_step(input_ids, 
#              input_mask, 
#              input_segment_ids, 
#              target_ids_, 
#              target_mask, 
#              target_segment_ids, 
#              target_ids, 
#              draft_mask, 
#              refine_mask,
#              step, 
#              create_summ):
#   with tf.device('/cpu:0'):
#     (draft_predictions, draft_attention_weights, 
#        refine_predictions, refine_attention_weights) = model(
#                                                              input_ids, input_mask, input_segment_ids, 
#                                                              target_ids_, target_mask, target_segment_ids, 
#                                                              False
#                                                              )
#   draft_summary_loss = loss_function(target_ids[:, 1:, :], draft_predictions, draft_mask)
#   refine_summary_loss = loss_function(target_ids[:, :-1, :], refine_predictions, refine_mask)
#   loss = draft_summary_loss + refine_summary_loss
#   loss = tf.reduce_mean(loss)
#   validation_loss(loss)
#   validation_accuracy(target_ids_[:, :-1], refine_predictions)  
#   if create_summ: 
#     rouge, bert = tf_write_summary(target_ids_[:, :-1], refine_predictions, step)  
#   else: 
#     rouge, bert = (1.0, 1.0)  
#   return (rouge, bert)

    

In [29]:

# for epoch in range(h_parms.epochs):
    
#   train_loss.reset_states()
#   train_accuracy.reset_states()
#   validation_loss.reset_states()
#   validation_accuracy.reset_states()
#   for (step, (input_ids, input_mask, input_segment_ids, target_ids_, target_mask, target_segment_ids)) in enumerate(train_dataset):
    
#       log.info(f"Start step {step+1}")
#       start=time.time()
      
#       draft_mask = tf.math.logical_not(tf.math.equal(target_ids_[:, 1:], 0))
#       refine_mask = tf.math.logical_not(tf.math.equal(target_ids_[:, :-1], 0))
#       target_ids = label_smoothing(tf.one_hot(target_ids_, depth=config.input_vocab_size))
#       grad_accum_flag = True if (step+1)%h_parms.accumulation_steps == 0 else False
#       target_x, refine_predictions=train_step(
#                   input_ids, 
#                   input_mask, 
#                   input_segment_ids, 
#                   target_ids_, 
#                   target_mask, 
#                   target_segment_ids, 
#                   target_ids, 
#                   draft_mask,
#                   refine_mask,
#                   grad_accum_flag
#                   )
#       if grad_accum_flag:
#         batch_run_check(
#                       step+1,  
#                       start, 
#                       train_summary_writer, 
#                       train_loss.result(), 
#                       train_accuracy.result(), 
#                       model
#                       )
#       eval_frequency = ((step+1) * h_parms.batch_size) % config.eval_after
#       if eval_frequency == 0:
#         predicted = (tokenizer.decode([i for i in tf.squeeze(tf.argmax(refine_predictions,axis=-1)) if i not in [101,102,0]]))
#         target = (tokenizer.decode([i for i in tf.squeeze(target_x) if i not in [101,102,0]]))
#         print(f'the golden summary is {target}')
#         print(f'the predicted summary is {predicted if predicted else "EMPTY"}')
#         ckpt_save_path = ck_pt_mgr.save()
#         (val_acc, val_loss, rouge_score, bert_score) = calc_validation_loss(
#                                                                             val_dataset, 
#                                                                             step+1, 
#                                                                             val_step, 
#                                                                             valid_summary_writer, 
#                                                                             validation_loss, 
#                                                                             validation_accuracy
#                                                                             )
        
#         latest_ckpt+=(step+1)
#         log.info(
#                 model_metrics.format(
#                                       step+1, 
#                                       train_loss.result(), 
#                                       train_accuracy.result(),
#                                       val_loss, 
#                                       val_acc,
#                                       rouge_score, 
#                                       bert_score
#                                     )
#                 )
#         log.info(evaluation_step.format(step+1, time.time() - start))
#         log.info(checkpoint_details.format(step+1, ckpt_save_path))
#         if not monitor_run(
#                           latest_ckpt, 
#                           ckpt_save_path, 
#                           val_loss, 
#                           val_acc, 
#                           bert_score, 
#                           rouge_score, 
#                           valid_summary_writer, 
#                           step+1):
#           break 

In [ ]:
total_steps = int(h_parms.epochs * (h_parms.accumulation_steps))
train_dataset = train_dataset.repeat(total_steps)
ck_pt_mgr, latest_ckpt = check_ckpt(file_path.checkpoint_path)
for (step, (input_ids, input_mask, input_segment_ids, target_ids_, target_mask, target_segment_ids)) in enumerate(train_dataset):
  # if step >= config.start_from_batch:
  with tf.device('/cpu:0'):
    log.info(f"Start step {step+1}")
    train_loss.reset_states()
    train_accuracy.reset_states()
    start=time.time()
    
    draft_mask = tf.math.logical_not(tf.math.equal(target_ids_[:, 1:], 0))
    refine_mask = tf.math.logical_not(tf.math.equal(target_ids_[:, :-1], 0))
    target_ids = label_smoothing(tf.one_hot(target_ids_, depth=config.input_vocab_size))
    grad_accum_flag = True if (step+1)%h_parms.accumulation_steps == 0 else False
    log.info(f'Start train_step with grad_accum_flag == {grad_accum_flag}')
    target_x, refine_predictions=train_step(
                  input_ids, 
                  input_mask, 
                  input_segment_ids, 
                  target_ids_, 
                  target_mask, 
                  target_segment_ids, 
                  target_ids, 
                  draft_mask,
                  refine_mask,
                  grad_accum_flag
                  )
  
  with tf.device('/cpu:0'):
    if grad_accum_flag:
      batch_run_check(
                    step+1,  
                    start, 
                    train_summary_writer, 
                    train_loss.result(), 
                    train_accuracy.result(), 
                    model
                    )

  predicted = (tokenizer.decode([i for i in tf.squeeze(tf.argmax(refine_predictions,axis=-1)) if i not in [101,102,0]]))
  target = (tokenizer.decode([i for i in tf.squeeze(target_x) if i not in [101,102,0]]))
  log.info(f'the golden summary is {target}')
  log.info(f'the predicted summary is {predicted if predicted else "EMPTY"}')
  ckpt_save_path = ck_pt_mgr.save()
  rouge_score = tf_write_summary(target_ids_[:, :-1], refine_predictions, step)

  log.info(evaluation_step.format(step+1, time.time() - start))
  log.info(checkpoint_details.format(step+1, ckpt_save_path))
  log.info(f'train_loss for {step+1} is {train_loss.result()}, train_accuracy: {train_accuracy.result()}, ROUGE score: {rouge_score}')
  

  latest_ckpt+=(step+1)
  
  


INFO:tensorflow:/home/jupyter/created_files/training_summarization_model_ckpt/cnn_dailymail/checkpoints/ckpt-429 restored
INFO:tensorflow:Start step 1
INFO:tensorflow:Start train_step with grad_accum_flag == False


Token indices sequence length is longer than the specified maximum sequence length for this model (1059 > 512). Running this sequence through the model will result in indexing errors


INFO:tensorflow:Calculate draft_predictions, refine_predictions
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Building:'Draft summary'
INFO:tensorflow:Building:'Refined summary'
INFO:tensorflow:draft_predictions shape: (None, None, 30522) - draft_mask shape: (None, None)
INFO:tensorflow:

In [ ]:
save_model_path = 'created_files/model'
tf.saved_model.save(
    model, save_model_path
)